# eICU Collaborative Research Database

# Notebook 1: Exploring the patient table

The aim of this notebook is to get set up with access to a demo version of the [eICU Collaborative Research Database](http://eicu-crd.mit.edu/). The demo is a subset of the full database, limited to ~1000 patients.

We begin by exploring the `patient` table, which contains patient demographics and admission and discharge details for hospital and ICU stays. For more detail, see: http://eicu-crd.mit.edu/eicutables/patient/

## Prerequisites

- If you do not have a Gmail account, please create one at http://www.gmail.com. 
- If you have not yet signed the data use agreement (DUA) sent by the organizers, please do so now to get access to the dataset.

## Load libraries and connect to the data

Run the following cells to import some libraries and then connect to the database.

In [0]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path

# Make pandas dataframes prettier
from IPython.display import display, HTML

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

Before running any queries, you need to first authenticate yourself by running the following cell. If you are running it for the first time, it will ask you to follow a link to log in using your Gmail account, and accept the data access requests to your profile. Once this is done, it will generate a string of verification code, which you should paste back to the cell below and press enter.

In [0]:
auth.authenticate_user()

We'll also set the project details.

In [0]:
project_id='sccm-datathon'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

# "Querying" our database with SQL

Now we can start exploring the data. We'll begin by running a simple query to load all columns of the `patient` table to a Pandas DataFrame. The query is written in SQL, a common language for extracting data from databases. The structure of an SQL query is:

```sql
SELECT <columns>
FROM <table>
WHERE <criteria, optional>
```

`*` is a wildcard that indicates all columns

# BigQuery

Our dataset is stored on BigQuery, Google's database engine. We can run our query on the database using some special ("magic") [BigQuery syntax](https://googleapis.dev/python/bigquery/latest/magics.html).

In [0]:
%%bigquery df

SELECT * FROM `sccm-datathon.team_5.glucose_cohort`

In [125]:
df.shape

(244532, 11)

In [126]:
df.head()

,patientunitstayid,LAB_NAME,labresult,labresultoffset,DAY,Glucose_max,Glucose_min,Glucose_mean,Glucose_std,Glucose_count,rn
0,146690,GLUCOSE,273.0,-98,1,292.0,94.0,196.576923,44.592531,26,1
1,146690,GLUCOSE,292.0,-43,1,292.0,94.0,196.576923,44.592531,26,2
2,146690,GLUCOSE,252.0,6,1,292.0,94.0,196.576923,44.592531,26,3
3,146690,GLUCOSE,213.0,70,1,292.0,94.0,196.576923,44.592531,26,4
4,146690,GLUCOSE,200.0,74,1,292.0,94.0,196.576923,44.592531,26,5


In [0]:
df.drop(labels=['Glucose_max', 'Glucose_min', 'Glucose_std', 'Glucose_count', 'Glucose_mean', 'rn'], axis=1, inplace=True)

In [128]:
df.head()

,patientunitstayid,LAB_NAME,labresult,labresultoffset,DAY
0,146690,GLUCOSE,273.0,-98,1
1,146690,GLUCOSE,292.0,-43,1
2,146690,GLUCOSE,252.0,6,1
3,146690,GLUCOSE,213.0,70,1
4,146690,GLUCOSE,200.0,74,1


In [129]:
df.sort_values(['patientunitstayid', 'DAY'], inplace=True)
df.head()

,patientunitstayid,LAB_NAME,labresult,labresultoffset,DAY
56110,141265,GLUCOSE,125.0,528,1
56111,141265,GLUCOSE,142.0,803,1
56112,141265,GLUCOSE,156.0,1193,1
56113,141265,GLUCOSE,155.0,1342,1
56114,141265,GLUCOSE,122.0,1660,2


#Calculate difference between successive glucose draws

In [130]:
df['delta_glucose'] = df['labresult'].diff()
df.head()

,patientunitstayid,LAB_NAME,labresult,labresultoffset,DAY,delta_glucose
56110,141265,GLUCOSE,125.0,528,1,NaN
56111,141265,GLUCOSE,142.0,803,1,17.0
56112,141265,GLUCOSE,156.0,1193,1,14.0
56113,141265,GLUCOSE,155.0,1342,1,-1.0
56114,141265,GLUCOSE,122.0,1660,2,-33.0


In [132]:
mask = df.patientunitstayid != df.patientunitstayid.shift(1)
df['delta_glucose'][mask] = np.nan
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,patientunitstayid,LAB_NAME,labresult,labresultoffset,DAY,delta_glucose
56110,141265,GLUCOSE,125.0,528,1,NaN
56111,141265,GLUCOSE,142.0,803,1,17.0
56112,141265,GLUCOSE,156.0,1193,1,14.0
56113,141265,GLUCOSE,155.0,1342,1,-1.0
56114,141265,GLUCOSE,122.0,1660,2,-33.0


# Calculate summmary statistics for differences between successive glucose draws

In [146]:
delta_glucose_max = pd.Series(df.groupby(['patientunitstayid', 'DAY']).agg('max').delta_glucose, name='delta_glucose_max')
delta_glucose_max.head()

patientunitstayid  DAY
141265             1      17.0
                   2      13.0
                   3      21.0
                   4      16.0
                   5      32.0
Name: delta_glucose_max, dtype: float64

In [147]:
delta_glucose_min = pd.Series(df.groupby(['patientunitstayid', 'DAY']).agg('min').delta_glucose, name='delta_glucose_min')
delta_glucose_min.head()

patientunitstayid  DAY
141265             1      -1.0
                   2     -33.0
                   3     -18.0
                   4     -22.0
                   5     -28.0
Name: delta_glucose_min, dtype: float64

In [148]:
delta_glucose_mean = pd.Series(df.groupby(['patientunitstayid', 'DAY']).agg('mean').delta_glucose, name='delta_glucose_mean')
delta_glucose_mean.head()

patientunitstayid  DAY
141265             1      10.0
                   2      -7.0
                   3      -0.8
                   4      -7.5
                   5       0.6
Name: delta_glucose_mean, dtype: float64

In [149]:
delta_glucose_std = pd.Series(df.groupby(['patientunitstayid', 'DAY']).agg('std').delta_glucose, name='delta_glucose_std')
delta_glucose_std.head()

patientunitstayid  DAY
141265             1       9.643651
                   2      19.165942
                   3      18.033303
                   4      16.441817
                   5      21.267346
Name: delta_glucose_std, dtype: float64

In [154]:
delta_glucose_count = pd.Series(df.groupby(['patientunitstayid', 'DAY']).agg('count').delta_glucose, name='delta_glucose_count')
delta_glucose_count.head()

patientunitstayid  DAY
141265             1      3
                   2      4
                   3      5
                   4      4
                   5      5
Name: delta_glucose_count, dtype: int64

In [157]:
df_lags = pd.concat([delta_glucose_min, delta_glucose_max, delta_glucose_mean, delta_glucose_std, delta_glucose_count], axis=1)
df_lags

delta_glucose_min  ...  delta_glucose_count
patientunitstayid DAY                     ...                     
141265            1                 -1.0  ...                    3
                  2                -33.0  ...                    4
                  3                -18.0  ...                    5
                  4                -22.0  ...                    4
                  5                -28.0  ...                    5
...                                  ...  ...                  ...
3353145           2                -71.0  ...                    4
                  3                -91.0  ...                    5
                  4                -50.0  ...                    4
                  5               -100.0  ...                    4
                  6                -27.0  ...                    4

[65379 rows x 5 columns]

In [0]:
df_lags.head().to_csv('test.csv')

In [163]:
!ls

adc.json  sample_data  test.csv


In [164]:
!head test.csv

patientunitstayid,DAY,delta_glucose_min,delta_glucose_max,delta_glucose_mean,delta_glucose_std,delta_glucose_count
141265,1,-1.0,17.0,10.0,9.643650760992955,3
141265,2,-33.0,13.0,-7.0,19.165942015286735,4
141265,3,-18.0,21.0,-0.8,18.033302526159762,5
141265,4,-22.0,16.0,-7.5,16.441816606851365,4
141265,5,-28.0,32.0,0.6,21.26734586167254,5


#Upload DataFrame to BigQuery

In [165]:
client = bigquery.Client(location='US', project='sccm-datathon')
dataset_ref = client.dataset('team_5')
table_ref = dataset_ref.table('glucose_cohort_deltas')
client.load_table_from_dataframe(df_lags, table_ref).result()